In [ ]:
import pandas as pd
import fancyimpute
from sklearn.preprocessing import StandardScaler


def merge_col(dropped):
    """

    :param dropped:
    :return:
    """
    indices = dropped.index[dropped["Max Ethanol % - Gasoline"].isnull()]
    dropped.loc[indices, "Max Ethanol % - Gasoline"] = dropped.loc[indices, "Max Biodiesel %"]
    dropped = dropped.drop("Max Biodiesel %", axis=1)
    dropped = dropped.rename(columns={"Max Ethanol % - Gasoline": "Max Fuel %"})

    merger = {"4Dr Pass Vol": "2Dr Pass Vol",
              "Htchbk Pass Vol": "2Dr Pass Vol",
              "4Dr Lugg Vol": "2Dr Lugg Vol",
              "Htchbk Lugg Vol": "2Dr Lugg Vol"}

    for item in merger:
        indices = dropped.index[dropped.loc[:, item].notnull()]
        dropped.loc[indices, merger[item]] = dropped.loc[indices, item]

    dropped = dropped.drop(list(merger), axis=1)
    dropped = dropped.rename(columns={"2Dr Lugg Vol": "Lugg Vol", "2Dr Pass Vol": "Pass Vol"})

    merge_from = "$ You Save over 5 years (amount saved in fuel costs over 5 years - on label)"
    merge_to = "$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label)"

    indices = dropped.index[dropped[merge_to].isnull()]
    dropped.loc[indices, merge_to] = -dropped.loc[indices, merge_from]
    dropped = dropped.drop(merge_from, axis=1)
    merged = dropped.rename(columns={merge_to: "spending over 5 years"})

    return merged


def impute_missing_values(merged):
    """

    :param merged:
    :return:
    """
    merged["Max Fuel %"] = merged["Max Fuel %"].fillna(0)

    # "Y" values were not entered
    merged["Fuel Cell Vehicle (Y or N)"] = merged["Fuel Cell Vehicle (Y or N)"].fillna("Y")

    # most frequent category is "N"
    merged['Unique Label?'] = merged['Unique Label?'].fillna("N")
    merged['Label Recalc?'] = merged['Label Recalc?'].fillna("N")

    fifteen_percent = int(merged.shape[0] * 0.15)
    merged = merged.dropna(axis=1, how='all', thresh=fifteen_percent)
    merged['GHG Rating (1-10 rating on Label)'] = merged['GHG Rating (1-10 rating on Label)'].astype(object)

    x_numeric = merged.select_dtypes(exclude=['object'])
    mice = fancyimpute.MICE(verbose=0)
    x_imputed = mice.complete(x_numeric)

    x_train_numeric2 = pd.DataFrame(x_imputed)
    x_train_numeric2.columns = x_numeric.columns

    x_train_categorical = merged.select_dtypes(include=['object'])
    complete = pd.concat([x_train_numeric2, x_train_categorical], axis=1)

    return complete


def drop_cols(raw_data):
    """

    :param raw_data:
    :return:
    """
    raw_data.columns = [x.strip() for x in raw_data.columns]

    remove_fe_related = [x for x in raw_data.columns if (('EPA' not in x) & ('FE' not in x) & ('CO2' not in x) &
                                                         ('smog' not in x) & ('Smog' not in x) & ('Guzzler' not in x))]

    redundant_features = ["Model Year", "Verify Mfr Cd", "Index (Model Type Index)", "Air Aspir Method",
                          "Trans", "Trans Desc", "Trans, Other", "Drive Desc", "Fuel Usage Desc - Conventional Fuel",
                          "Fuel Unit - Conventional Fuel", "Fuel Unit Desc - Conventional Fuel", "Carline Class",
                          "Descriptor - Model Type (40 Char or less)", "Car/Truck Category - Cash for Clunkers Bill.",
                          "Comments - Mfr Eng Cnfg", "Fuel Metering Sys Desc", "Var Valve Timing Desc",
                          "Cyl Deact Desc", "Var Valve Lift Desc", "Release Date", "Relabel Desc",
                          "Stop/Start System (Engine Management System)  Description", "Model Type Desc (MFR entered)"]

    dropped = raw_data[remove_fe_related]
    dropped = dropped.drop(redundant_features, axis=1)
    dropped = dropped.dropna(axis=1, how='all')

    return dropped


def data_preprocessiong(raw_data):
    """

    :param raw_data:
    :return:
    """
    dropped = drop_cols(raw_data)
    merged = merge_col(dropped)
    complete = impute_missing_values(merged)

    return complete


def scale(x_train_num, x_test_num):
    """

    :param x_train_num:
    :param x_test_num:
    :return:
    """
    scaler = StandardScaler()
    scaler.fit(x_train_num)
    x_train_numeric_scaled = scaler.transform(x_train_num)
    x_test_numeric_scaled = scaler.transform(x_test_num)

    x_train_num_final = pd.DataFrame(x_train_numeric_scaled)
    x_train_num_final.columns = x_train_num.columns

    x_test_num_final = pd.DataFrame(x_test_numeric_scaled)
    x_test_num_final.columns = x_test_num.columns

    return x_train_num_final, x_test_num_final


def main():
    fe15 = pd.read_excel('data/2015 FE Guide-for DOE-Mobility Ventures only-OK to '
                         'release-no-sales-4-27-2017Mercedesforpublic.xlsx')
    fe16 = pd.read_excel('data/2016 FE Guide for DOE-OK to release-no-sales-4-27-2017Mercedesforpublic.xlsx')
    fe17 = pd.read_excel('data/2017 FE Guide for DOE-release dates before 9-20-2017-no sales-9-19-2017MercedesCadillacforpublic.xlsx')

    fe18 = pd.read_excel('data/2018 FE Guide for DOE-release dates before 2-10-2018-no-sales-2-9-2018public.xlsx')

    training = pd.concat([fe15, fe16, fe17], axis=0, ignore_index=True)

    x_train = data_preprocessiong(training)
    y_train = training["Comb Unrd Adj FE - Conventional Fuel"]
    x_test = data_preprocessiong(fe18)
    y_test = fe18["Comb Unrd Adj FE - Conventional Fuel"]

    x_train_num = x_train.select_dtypes(exclude=['object'])
    x_train_cat = x_train.select_dtypes(include=['object'])
    x_test_num = x_test.select_dtypes(exclude=['object'])
    x_test_cat = x_test.select_dtypes(include=['object'])

    x_train_num_final, x_test_num_final = scale(x_train_num, x_test_num)

    x_train_scaled = pd.concat([x_train_num_final, x_train_cat], axis=1)
    x_test_scaled = pd.concat([x_test_num_final, x_test_cat], axis=1)

    y_train.to_csv("y_train.csv", header=["FE"], index=False)
    y_test.to_csv("y_test.csv", header=["FE"], index=False)
    pd.get_dummies(x_train).to_csv("x_train.csv", index=False)
    pd.get_dummies(x_test).to_csv("x_test.csv", index=False)
    pd.get_dummies(x_train_scaled).to_csv("x_train_scaled.csv", index=False)
    pd.get_dummies(x_test_scaled).to_csv("x_test_scaled.csv", index=False)


if __name__ == "__main__":
    main()